In [ ]:
# BASIC 3D beamforming code: Delay and sum beamformer simulation for expensive rig

import numpy as np
import matplotlib.pyplot as plt
import pyroomacoustics as pra

### Effect of source frequency ###

# Create a shoebox room of similar size to the theatre
room1 = pra.ShoeBox([10, 20, 10])

# Add a source on stage
room1.add_source([5, 18, 3])

# Create a linear array beamformer with 8 cardioid microphones
# with angle 0 degrees and inter mic distance 10 cm
R = pra.linear_3D_array(center=[5, 15, 3], M=8, phi=0, d=0.05)
# mic_array = pra.beamforming.MicrophoneArray(R, room1.fs, directivity)
mic_array = pra.Beamformer(R, room1.fs)
room1.add_microphone_array(mic_array)

# Now compute the delay and sum weights for the beamformer
room1.mic_array.rake_delay_and_sum_weights(room1.sources[0][:1])

# plot the room and resulting beamformer
room1.plot(freq=[1000, 2000, 4000, 8000], img_order=0)
plt.title('Effect of Source Frequency')
plt.show()

# 3D Algorithm Simulation

Includes a variety of situations to test algorithms in a simulated theatre.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pyroomacoustics as pra

### Build Theatre Environment

Create polygonal room with a raised stage to simulate in.

In [ ]:
# build theatre env here